# How to use this notebook
Use this notebook to understand the quality and performance of your synthetic or augmented data on downstream machine learning classification tasks. 



# Installation
Install Gretel Client to use both Gretel's synthetic models as well as the Gretel Evaluate Classification model. You'll have to get your API key from the Gretel console dashboard to configure your session. 

In [ ]:
!pip install -U gretel-client

In [ ]:
from gretel_client import configure_session

configure_session(endpoint="https://api-dev.gretel.cloud", api_key="prompt", cache="yes")


 # Option 1: Generate synthetic data, then evaluate the synthetic data on classifiers against real-world data
 First, we'll generate synthetic data using a publicly available bank marketing dataset, which predicts whether a client will subscribe a term deposite (prediction: yes/no in column "y"). We'll use Gretel's ACTGAN model to train on the real-world and generate the synthetic data.
 
 To use the Gretel Evaluate Classification model, you must indicate the target column. Optionally, you can change the test-holdout amount, which is a float indicating the amount of real-world data you want to use as a holdout for testing the downstream classifiers. Youc an also optionally select which classifiers to use and which metric to optimize for. 

In [ ]:
#### SUPPORTED MODELS AND METRICS ####

classification_models = [
    "lr", 
    "knn", 
    "nb", 
    "dt", 
    "svm", 
    "rbfsvm", 
    "gpc", 
    "mlp", 
    "ridge", 
    "rf", 
    "qda", 
    "ada", 
    "gbc", 
    "lda", 
    "et", 
    "xgboost", 
    "lightgbm", 
    "dummy"
]

classification_metrics = [
    "acc",
    "auc",
    "recall",
    "precision",
    "f1",
    "kappa",
    "mcc"
]

In [ ]:
# Create a project with a name that describes this use case
from gretel_client.projects import create_or_get_unique_project

project = create_or_get_unique_project(name="bank-marketing-synthetic-data-downstream-classification-evaluation")

In [ ]:
from gretel_client.helpers import poll
from gretel_client.projects.models import read_model_config

# We'll import the bank_marketing_small dataset from Gretel's public S3 bucket
# You can modify this to select a dataset of your choice
dataset_path = ""

# We will modify the config for Gretel synthetic models to add an extra downstream Evaluate model and task
# Uncomment the additional params to change from defaults.
config = read_model_config("synthetics/tabular-actgan")

config["models"][0]["synthetics"]["evaluate"] = {
    # Available downstream tasks are "classification" or "regression"
    "task": "classification",
    # Set to the target you wish to predict -- Change this if you try a different data set!
    "target": "V3",  # marital status, use a categorical column for classification
    # "holdout": 0.2,  # default holdout value
    # "models": classification_models,  # default set of models
    # "metric": "acc",  # default metric used for sorting results, choose one
}


In [ ]:
# Train and run the model
model = project.create_model_obj(
    model_config=config, 
    data_source=dataset_path
)

model.submit_cloud()

poll(model)

# Save all artifacts
model.download_artifacts("/tmp")

# Option 2: BYO sythetic or augmented data to evaluate downstream classification against real-world data
Already have your synthetic or augmented data? You can use your own CSV or JSONL data files in the Gretel Evaluate Classification model. 

In [ ]:
# Use Evaluate SDK using your custom config
from gretel_client.evaluation.downstream_classification_report import DownstreamClassificationReport

# Params
# This is the synthetic data, REQUIRED for evaluate model
data_source = "bank_marketing_synthetic_data.csv"

# This is the real-world data, REQUIRED for evaluate model
ref_data = "bank_marketing_small.csv"

# Target to predict, REQUIRED for evaluate model
target = 'y'  # prediction field for whether client will subscribe to a bank term deposit

# Default holdout value
# test_holdout = 0.2

# Supply a subset if you do not want all of these, default is to use all of them
# models = classification_models

# Metric to use for ordering results, defaults are "acc" (Accuracy) for classification, "r2" (R2) for regression.
# metric = "acc"

# Create a downstream classification report
evaluate = DownstreamClassificationReport(
    # project=None,  # Create a temp project
    target=target, 
    data_source=data_source, 
    ref_data=ref_data,
    # holdout=test_holdout,
    # models=models,
    # metric=metric,
    # output_dir=None,
    # runner_mode="cloud",
)


In [ ]:

evaluate.run() # this will wait for the job to finish

# This will return the full report JSON details.
evaluate.as_dict

# This will return the full HTML contents of the report.
evaluate.as_html

# Returns a dictionary representation of the top level report scores.
evaluate.peek()